<img align="center" style="max-width: 1000px" src="https://github.com/HSG-AIML-Teaching/GSERM2024-Lab/blob/main/lab_05/banner.png?raw=1">

<img align="right" style="max-width: 200px; height: auto" src="https://github.com/HSG-AIML-Teaching/GSERM2024-Lab/blob/main/lab_05/hsg_logo.png?raw=1">

##  Lab 05 - Convolutional Neural Networks (CNNs)

GSERM Summer School 2024, Deep Learning: Fundamentals and Applications, University of St. Gallen

The lab environment is based on Jupyter Notebooks (https://jupyter.org), which provide an interactive platform for performing a variety of statistical evaluations and data analyses. In this lab, we will learn how to enhance vanilla **Artificial Neural Networks (ANNs)** using `PyTorch` to classify even more complex images. We will explore a special type of deep neural network known as **Convolutional Neural Networks (CNNs)** to achieve this. CNNs leverage the hierarchical pattern in data, allowing them to assemble more complex patterns from smaller, simpler ones. This hierarchical structure enables CNNs to learn a set of discriminative features and subsequently utilize these learned patterns to classify the content of an image.

The history of CNNs is rich and exhibits pivotal contributions from researchers like *Yann LeCun*, who developed the first practical CNN, known as **LeNet**, in the late 1980s. CNNs have since become a cornerstone in deep learning, significantly advancing the capabilities of image recognition and classification.

In this lab, we will use the `PyTorch` library to implement and train a CNN-based neural network. Our network will be trained on tiny images from the **CIFAR-10** dataset, which includes aeroplanes, cars, birds, cats, deer, dogs, frogs, horses, ships, and trucks. We will utilize the learned CNN model to classify previously unseen images into these distinct categories upon successful training.

The figure below illustrates a high-level view of the machine learning process we aim to establish in this lab.

<img align="center" style="max-width: 900px" src="https://github.com/HSG-AIML-Teaching/GSERM2024-Lab/blob/main/lab_05/splash.png?raw=1">

## 2. Setup of the Jupyter Notebook Environment

Similar to the previous labs, we need to import several Python libraries that facilitate data analysis and visualization. We will primarily use `PyTorch`, `NumPy`, `Scikit-learn`, `Matplotlib`, `Seaborn`, and a few utility libraries throughout this lab:

In [ ]:
# =======================
# 🧠 MLOps CIFAR10 Training - Local Setup (Neptune + Optuna)
# =======================

# ---- Install Dependencies ----
!pip install -q neptune optuna neptune-optuna torchvision torch scikit-learn matplotlib seaborn

# ---- Imports ----
import os
from datetime import datetime
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split, DataLoader
import numpy as np
import neptune.new as neptune
import optuna
from neptune.integrations.optuna import NeptuneCallback

from sklearn.metrics import classification_report

# ---- Neptune Init ----
run = neptune.init_run(
project="myproject",
    api_token="mytoken"  # Replace this with your token
)

# ---- CIFAR10 Dataset ----
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Split dataset into train, val, test
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
rest_size = len(dataset) - train_size - val_size
train_set, val_set, _ = random_split(dataset, [train_size, val_size, rest_size])
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

run["data/name"] = "CIFAR-10"
run["data/source"] = "torchvision.datasets"
run["data/train_size"] = len(train_set)
run["data/val_size"] = len(val_set)
run["data/test_size"] = len(test_dataset)
run["data/classes"] = dataset.classes
run["data/transform"] = str(transform)

# ---- Model Definition ----
class CIFAR10Net(nn.Module):
    def __init__(self):
        super(CIFAR10Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.logsoftmax(self.fc3(x))
        return x

# ---- Objective Function for Optuna ----
def objective(trial):
    # Suggest hyperparameters
    lr = trial.suggest_categorical("lr", [0.0001, 0.001, 0.005, 0.01])
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
    optimizer_name = trial.suggest_categorical("optimizer", ["SGD", "Adam"])

    # Dataloaders
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=1000)

    model = CIFAR10Net()
    criterion = nn.NLLLoss()

    optimizer = optim.SGD(model.parameters(), lr=lr) if optimizer_name == "SGD" else optim.Adam(model.parameters(), lr=lr)

    # Train
    model.train()
    for epoch in range(20):
        epoch_loss = 0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        run["training/epoch_loss"].append(epoch_loss / len(train_loader))

    # Validation
    model.eval()
    val_loss = 0
    correct = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()

    val_acc = correct / len(val_set)
    run["validation/accuracy"].append(val_acc)
    return val_loss

# ---- Optuna Setup ----
study = optuna.create_study(direction="minimize")
neptune_callback = NeptuneCallback(run=run)

study.optimize(objective, n_trials=24, callbacks=[neptune_callback])

# ---- Save Best Model ----
best_params = study.best_params
run["best_hyperparams"] = best_params

# ---- Retrain on full train + val ----
combined_set = torch.utils.data.ConcatDataset([train_set, val_set])
combined_loader = DataLoader(combined_set, batch_size=best_params["batch_size"], shuffle=True)

final_model = CIFAR10Net()
criterion = nn.NLLLoss()
optimizer = optim.SGD(final_model.parameters(), lr=best_params["lr"]) if best_params["optimizer"] == "SGD" else optim.Adam(final_model.parameters(), lr=best_params["lr"])

final_model.train()
for epoch in range(20):
    total_loss = 0
    for inputs, labels in combined_loader:
        optimizer.zero_grad()
        outputs = final_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    run["final_training/epoch_loss"].append(total_loss / len(combined_loader))

# ---- Save Final Model ----
os.makedirs("models", exist_ok=True)
torch.save(final_model.state_dict(), "models/BEST_CNN.pth")
run["model/best_model"].upload("models/BEST_CNN.pth")

# ---- Evaluate on Test ----
final_model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = final_model(inputs)
        preds = outputs.argmax(dim=1)
        all_preds.extend(preds.tolist())
        all_labels.extend(labels.tolist())

print("Test Classification Report:")
print(classification_report(all_labels, all_preds, target_names=dataset.classes))

run.stop()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.9/487.9 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00

[neptune] [warning] NeptuneDeprecationWarning: You're importing the Neptune client library via the deprecated `neptune.new` module, which will be removed in a future release. Import directly from `neptune` instead.
[neptune] [warning] NeptuneWarning: By default, these monitoring options are disabled in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', 'capture_hardware_metrics'. You can set them to 'True' when initializing the run and the monitoring will continue until you call run.stop() or the kernel stops. NOTE: To track the source files, pass their paths to the 'source_code' argument. For help, see: https://docs-legacy.neptune.ai/logging/source_code/


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/luisaellamueller/mlops/e/MLOPS-31


100%|██████████| 170M/170M [00:03<00:00, 43.0MB/s]
[neptune] [warning] NeptuneUnsupportedType: You're attempting to log a type that is not directly supported by Neptune (<class 'list'>).
        Convert the value to a supported type, such as a string or float, or use stringify_unsupported(obj)
        for dictionaries or collections that contain unsupported values.
        For more, see https://docs-legacy.neptune.ai/help/value_of_unsupported_type
[I 2025-05-12 09:50:11,727] A new study created in memory with name: no-name-baea41c6-c63a-4a6b-aa6e-6ca334fb0f62
[I 2025-05-12 09:55:27,783] Trial 0 finished with value: 12.146215677261353 and parameters: {'lr': 0.01, 'batch_size': 32, 'optimizer': 'SGD'}. Best is trial 0 with value: 12.146215677261353.
[W 2025-05-12 09:55:28,462] Param lr unique value length is less than 2.
[W 2025-05-12 09:55:28,463] Param optimizer unique value length is less than 2.
[W 2025-05-12 09:55:28,464] Param batch_size unique value length is less than 2.
[W 2025-